In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp -r -v drive/My\ Drive/ThaiLanguageModel/* .
!rsync -a --progress --exclude="dataset" drive/My\ Drive/ThaiLanguageModel/* .

sending incremental file list
README.md
             19 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=5/6)
character_copus.txt
            259 100%    0.72kB/s    0:00:00 (xfr#2, to-chk=4/6)
data_gen.py
          2,559 100%    3.13kB/s    0:00:00 (xfr#3, to-chk=3/6)
model.ipynb
         74,896 100%   58.09kB/s    0:00:01 (xfr#4, to-chk=2/6)
model.py
              0 100%    0.00kB/s    0:00:00 (xfr#5, to-chk=1/6)
wiki_prepare.py
          2,019 100%    4.59kB/s    0:00:00 (xfr#6, to-chk=0/6)


In [0]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/thwiki-words
!cp drive/My\ Drive/ThaiLanguageModel/dataset/thwiki-words.zip dataset/thwiki-words.zip
!unzip dataset/thwiki-words.zip -d dataset

In [1]:
!pip install ujson
!pip install pythainlp

     |████████████████████████████████| 194kB 1.1MB/s eta 0:00:01
  Created wheel for ujson: filename=ujson-1.35-cp37-cp37m-linux_x86_64.whl size=69600 sha256=effde6ec631d01ff6dc88b2e273b4d1064dae9e3b031532d02f2fc16ff7f6144
  Stored in directory: /home/porlolicon/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
     |████████████████████████████████| 11.0MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 10.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/20/95/d23071d0992dabcb61c948fb118a90683193befc88c23e745b050a29e7db/marisa-trie-0.7.5.tar.gz
     |████████████████████████████████| 61kB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 10.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 10.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4b

In [6]:
%load_ext autoreload
%autoreload 2

In [30]:
import numpy as np
from keras.layers import CuDNNLSTM, Bidirectional, RepeatVector, Dense, TimeDistributed, Activation, Input
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from pathlib import Path
import data_gen

In [8]:
NUM_VOCAB = data_gen.NUM_VOCAB
length = 32
batch_size = 32
train_gen = data_gen.sentence_generator(Path('dataset/thwiki-words'),
                                        length,
                                        batch_size,
                                        adversarial=data_gen.random_blank(0.05))

In [4]:
model = load_model('weight/test.h5')

W1126 00:16:01.211741 139818380121920 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W1126 00:16:03.606885 139818380121920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1126 00:16:06.450649 139818380121920 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [4]:
model = Sequential()
activation = 'relu'
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(length, NUM_VOCAB)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))


# model.add(RepeatVector(length))


model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(32, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Activation(activation))
model.add(TimeDistributed(Dense(NUM_VOCAB)))
model.add(Activation('softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

W1124 22:20:31.104708 140365793371968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 32, 128)           77824     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 128)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           99328     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 128)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32, 64)            41472     
_________________________________________________________________
activation_3 (Activation)    (None, 32, 64)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 32, 64)           

In [7]:
model.fit_generator(train_gen,
                    steps_per_epoch=4096,
                    initial_epoch=41,
                    epochs=100)

Epoch 42/100
 433/4096 [==>...........................] - ETA: 1:40 - loss: 0.0547 - acc: 0.9860

KeyboardInterrupt: 

In [9]:
model.save('weight/test.h5')

In [43]:
ads_func = data_gen.random_blank(0.1)

In [94]:
text = 'อยากอกหักแต่อุปสรรคอยู่ที่หน้าตา'
text = ads_func(text)
print('input:', text)
encoded = data_gen.encode_text(text, 32)
pred = model.predict(np.expand_dims(encoded, axis=0))[0]
pred_text = data_gen.decode_text(pred)
print('output:', pred_text)

input: อยากอกหักแต$อุปสรรคอยู$ที่ห$้าตา
output: อยากอกหักแต่อุปสรรคอยู่ที่หน้าตา
